In [1]:
import os
import itertools
from itertools import groupby
from numpy import *
import operator
import numpy as np
import pandas as pd
import math
from statistics import mean
from pandas import *  
import glob

In [13]:
refs        ='/home/alva/Documents/lncRNAs_project_2020/Analysis_2021/Tcga_R/GDCdata/TCGA-BRCA/harmonized/Transcriptome_Profiling/Gene_Expression_Quantification/r_data/reference/Homo_sapiens.GRCh38.97.chr.txt'
ref         = pd.read_csv(refs, sep="\t", header=0,skipinitialspace=True)
lncRNAs_ref = ref.query('biotype == "lncRNA"')
ref

,chr,start,end,gene_id,gene_sym,biotype
0,1,11869,14409,ENSG00000223972,DDX11L1,transcribed_unprocessed_pseudogene
1,1,14404,29570,ENSG00000227232,WASH7P,unprocessed_pseudogene
2,1,17369,17436,ENSG00000278267,MIR6859-1,miRNA
3,1,29554,31109,ENSG00000243485,MIR1302-2HG,lncRNA
4,1,30366,30503,ENSG00000284332,MIR1302-2,miRNA
...,...,...,...,...,...,...
60612,KI270711.1,4612,29626,ENSG00000271254,AC240274.1,protein_coding
60613,KI270713.1,21861,22024,ENSG00000275405,RF00003,snRNA
60614,KI270713.1,30437,30580,ENSG00000275987,RF00003,snRNA
60615,KI270713.1,31698,32528,ENSG00000277475,AC213203.2,protein_coding


In [55]:
debraca = '/home/alva/Documents/lncRNAs_project_2020/Analysis_2021/Tcga_R/GDCdata/TCGA-BRCA/harmonized/Transcriptome_Profiling/Gene_Expression_Quantification/r_data_new/de'
de      = pd.read_csv(debraca, sep="\t", header=0,skipinitialspace=True)
de.rename(columns={'adj.P.Val':'adj_pval'}, inplace=True)

In [5]:
path         = '/home/alva/Documents/lncRNAs_project_2020/Analysis_2021/Tcga_R/GDCdata/TCGA-BRCA/harmonized/Transcriptome_Profiling/Gene_Expression_Quantification/r_data_new/de' 
files        = os.listdir(path)
results      = [os.path.join(path,i) for i in files if i.startswith('TCGA')]

['/home/alva/Documents/lncRNAs_project_2020/Analysis_2021/Tcga_R/GDCdata/TCGA-BRCA/harmonized/Transcriptome_Profiling/Gene_Expression_Quantification/r_data_new/de/TCGA-THCA_DE',
 '/home/alva/Documents/lncRNAs_project_2020/Analysis_2021/Tcga_R/GDCdata/TCGA-BRCA/harmonized/Transcriptome_Profiling/Gene_Expression_Quantification/r_data_new/de/TCGA-LIHC_DE',
 '/home/alva/Documents/lncRNAs_project_2020/Analysis_2021/Tcga_R/GDCdata/TCGA-BRCA/harmonized/Transcriptome_Profiling/Gene_Expression_Quantification/r_data_new/de/TCGA-COAD_DE',
 '/home/alva/Documents/lncRNAs_project_2020/Analysis_2021/Tcga_R/GDCdata/TCGA-BRCA/harmonized/Transcriptome_Profiling/Gene_Expression_Quantification/r_data_new/de/TCGA-ESCA_DE',
 '/home/alva/Documents/lncRNAs_project_2020/Analysis_2021/Tcga_R/GDCdata/TCGA-BRCA/harmonized/Transcriptome_Profiling/Gene_Expression_Quantification/r_data_new/de/TCGA-BRCA_DE',
 '/home/alva/Documents/lncRNAs_project_2020/Analysis_2021/Tcga_R/GDCdata/TCGA-BRCA/harmonized/Transcriptome_Pr

In [6]:
def DEfilter(df):
    Up_regulted = df.query('logFC >= 0.58 and adj_pval <= 0.05')
    Down_regulated = df.query('logFC <= -0.58 and adj_pval <= 0.05')
    DE = pd.concat([Up_regulted, Down_regulated])
    return DE

In [26]:
for filename in results:
    df = pd.read_csv(filename, sep='\t',header=0,skipinitialspace=True)
    df.rename(columns={'adj.P.Val':'adj_pval'}, inplace=True)
    df = df.drop(['original_ensembl_gene_id','Row.names','t','B'], axis = 1) 
    df.rename(columns={'ensembl_gene_id':'gene_id'},inplace=True)
    DE = DEfilter(df)
    lncRNA    = pd.merge(DE,lncRNAs_ref[["chr","start","end","gene_id","gene_sym"]],on='gene_id')
    Annon_all = pd.merge(DE,ref[["chr","start","end","gene_id","gene_sym"]],on='gene_id')
    name      = os.path.basename(os.path.normpath(filename))
    dirname   = os.path.dirname(os.path.normpath(filename))
    lncRNA.to_csv(dirname +'/lncRNA_0.58_' + name, sep='\t',index=False)
    Annon_all.to_csv(dirname +'/Annon_0.58_' + name, sep='\t',index=False)

In [25]:
df.iloc[1:5,]

,ensembl_gene_id,external_gene_name,logFC,AveExpr,P.Value,adj_pval,TCGA-EL-A3ZT-11A-13R-A23N-07,TCGA-BJ-A4O8-01A-11R-A250-07,TCGA-H2-A3RI-01A-11R-A220-07,TCGA-ET-A3BQ-01B-11R-A206-07,...,TCGA-EL-A3H7-01A-11R-A21D-07,TCGA-DJ-A3UT-01A-11R-A22L-07,TCGA-L6-A4ET-01A-11R-A250-07,TCGA-EL-A3T2-11A-11R-A22L-07,TCGA-EL-A3T1-01A-11R-A22L-07,TCGA-FE-A237-01A-11R-A16R-07,TCGA-EM-A3FJ-01A-11R-A21D-07,TCGA-EL-A3ZO-11A-12R-A23N-07,TCGA-EL-A3D4-01A-11R-A19O-07,TCGA-FE-A234-01A-11R-A14Y-07
1,ENSG00000000005,TNMD,-1.131236,-4.626060,1.406871e-09,5.087511e-09,-4.791540,-5.290590,-4.146977,-5.715602,...,-4.692280,-3.700297,-5.331899,-2.501249,-7.261110,-4.187012,-3.996714,-0.846589,-2.328030,-7.131660
2,ENSG00000000419,DPM1,-0.063570,4.914410,1.094661e-01,1.409930e-01,4.708705,4.686213,4.996265,4.897573,...,5.064277,4.967298,4.796096,4.940699,4.974605,4.569306,4.727800,5.288611,4.761107,5.214577
3,ENSG00000000457,SCYL3,-0.113695,4.084091,7.300739e-03,1.156434e-02,4.448298,3.987860,4.510838,3.996925,...,4.104085,3.912404,4.198833,4.749333,4.082631,3.972139,4.368151,3.770136,3.769448,4.153164
4,ENSG00000000460,C1orf112,0.294624,1.893186,4.367210e-05,9.409411e-05,1.533991,2.602206,2.164897,2.019107,...,2.418334,0.984929,1.427435,2.057018,2.505419,2.391496,1.907503,1.653584,1.237824,2.378115


In [54]:
DE        = DEfilter(de)
lncRNA    = pd.merge(DE,lncRNAs_ref,on='gene_id')
name      = os.path.basename(os.path.normpath(debraca))
dirname   = os.path.dirname(os.path.normpath(debraca))
lncRNA.to_csv(dirname +'/lncRNA_0.58_' + name, sep='\t',index=False)